In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from timeit import timeit
from datetime import date, timedelta
import businessdate as bd
from businessdate import BusinessDate, BusinessRange
from businessdate.daycount import *
import dcf
from tabulate import tabulate
from yieldcurves.parametric import NelsonSiegelSvensson, \
    NelsonSiegelSvenssonShortRate, download_ecb
from yieldcurves.tools import ascii_plot, lin, plot, AlgebraCurve
from yieldcurves import *
from yieldcurves.datecurve import DateCurve
from yieldcurves.interpolation import linear, constant
from yieldcurves.compounding import *
from yieldcurves.yieldcurves import Df, YieldCurve

TODO:
* test compounding [OK]
* test interpolation [OK]
* test parametric [OK]
* test curves [OK]
* test datecurve [ ]

In [4]:
from yieldcurves.tools import inverse

inverse((-5.4)**3, lambda x: x**3)

-5

In [ ]:

today = bd.BusinessDate()
curve = AlgebraCurve()
a = DateCurve(curve, origin=today, yf=get_act_act)
d = today + '1y2m'
f = a.year_fraction(d)
p = a.inverse(f)
g = a.year_fraction(p)
today, d, f, p, g

In [ ]:
d, a = (BusinessDate(20241103), DateCurve(AlgebraCurve(), origin=BusinessDate(20240630), yf=get_act_360))

f = a.year_fraction(d)
p = a.inverse(f)
g = a.year_fraction(p)

d == p, d, f, p, g

In [ ]:
d, a = (1.9, DateCurve(AlgebraCurve(), origin=0.25))

def yf(x):
    return a.year_fraction(a.origin + x / a.DAYS_IN_YEAR)
f = a.year_fraction(d)
i = inverse(f, yf) / a.DAYS_IN_YEAR
d, f, i, a.origin + i

In [5]:
d, a = (BusinessDate(20241103), DateCurve(AlgebraCurve(), origin=BusinessDate(20240630), yf=get_act_360))
d, a.inverse(a.year_fraction(d))

(BusinessDate(20241103), BusinessDate(20241103))

In [ ]:

def yf(x):
    return a.year_fraction(a.origin + timedelta(x))

f = a.year_fraction(d)
i = timedelta(inverse(f, yf, step=256))

d, f, i, a.origin + i, a.inverse(f)

In [ ]:
for yf  in (.2, 1.2, 2.2, 5., 10., 50, 100):
    aa = timeit(lambda: a._inverse(yf), number=100)
    # b = timeit(lambda: a._inverse1(yf, step=1), number=100)
    bb = timeit(lambda: a._inverse1(yf), number=100)
    bbb = timeit(lambda: a._inverse1(yf, step=4096), number=100)
    bbbb = timeit(lambda: a._inverse1(yf, step=1024), number=100)
    c = timeit(lambda: a._inverse2(yf), number=100)
    print(yf, aa, bb, bbb, bbbb, c)

In [ ]:


f = a.year_fraction(d)
p = a.inverse(f)
g = a.year_fraction(p)

d == p, d, f, p, g

In [ ]:
X = lin(0.000001, 20.05, 0.05)

v = AlgebraCurve(nss)
f = YieldCurve(v)

g = YieldCurve.from_marginal_probs(f.marginal)
h = YieldCurve.from_marginal_pd(f.marginal_pd)

plot(X, original=f-v, marginal=g-v)#, m_pd=h-v)

In [ ]:
YieldCurve.from_cash_rates(YieldCurve(0.01).cash)(0.25)

In [ ]:

X = lin(0.0001, 20.05, 0.01)
c = AlgebraCurve(f.cash)
plot(X, spot=YieldCurve.from_cash_rates(f.cash)-v, cash=YieldCurve.from_cash_rates(c).cash-c)

In [ ]:
_params = {
    'beta0': 1,
    'beta1': 2,
    'beta2': 3,
    'beta3': 6,
    'tau1': 1,
    'tau2': 10,
}
nss = NelsonSiegelSvensson(**_params)
l_nss = linear([1, 2, 3, 5, 7, 10, 15], nss)
l_nss

# Plot

In [ ]:
X = lin(0.05, 20.05, 0.05)
a = AlgebraCurve(0.02)
a = AlgebraCurve(nss)
b = YieldCurve(a, frequency=4, cash_frequency=4, swap_frequency=2)
plot(X, curve=a, yield_curve=b)

In [ ]:
c = YieldCurve.from_prices(b.price)
d = YieldCurve.from_short_rates(b.short)
plot(X, spot=a-b, price=a-c, short=a-d)

In [ ]:
e = YieldCurve.from_df(b.df)
f = YieldCurve.from_zero_rates(b.zero, compounding_frequency=b.frequency)
g = YieldCurve.from_cash_rates(b.cash, compounding_frequency=b.cash_frequency)
h = YieldCurve.from_swap_rates(b.swap, compounding_frequency=b.swap_frequency)
plot(X, df=a-e, zero=a-f, cash=a-g, swap=a-h)

In [ ]:
i = YieldCurve.from_probs(b.prob)
j = YieldCurve.from_intensities(b.intensity)
k = YieldCurve.from_hazard_rates(b.hz)
l = YieldCurve.from_pd(b.pd)
plot(X, prob=a-i, intensity=a-j, hz=a-k, pd=a-l)

In [ ]:
m = YieldCurve.from_marginal_probs(b.marginal)
n = YieldCurve.from_marginal_pd(b.marginal_pd)
plot(X, marginal=a-m, marginal_pd=a-m)

In [ ]:
params = {
    'beta0': 1.0138959988,
    'beta1': 1.836312606,
    'beta2': 2.9874138836,
    'beta3': 4.8105550065,
    'tau1': 0.7389058665,
    'tau2': 12.0362372437
}

In [ ]:
# float
today = 0
times = 1, 5, 10
dates = [today + t for t in times]
values = [0.01, 0.03, 0.02]
times = DateCurve.dyf(today, dates)

l = linear(times, values)
c = YieldCurve.from_spot_rates(l)
d = DateCurve(c, origin=today)

Df(d)(9), d.df(9)

In [ ]:
# BusinessDate
today = bd.BusinessDate()
times = "1y", "5y", "10y"
dates = [today + t for t in times]
values = [0.01, 0.03, 0.02]
times = DateCurve.dyf(today, dates)

l = linear(times, values)
c = YieldCurve.from_spot_rates(l)
d = DateCurve(c, origin=today)

#d.domain
Df(d)(today + '9y'), d.df(today + '9y')

In [ ]:
DateCurve.basedate = today# + '3m'
dd = DateCurve.from_interpolation(dates, values, origin=today, interpolation='constant', curve_type='from_spot_rates')
tuple(dd)

In [ ]:
dd.inverse(-2.5), today - '2y6m'

In [ ]:
# datetime.date
today = bd.BusinessDate()
times = "1y", "5y", "10y"
dates = [(today + t).to_date() for t in times]
values = [0.01, 0.03, 0.02]
times = DateCurve.dyf(today.to_date(), dates)

l = linear(times, values)
c = YieldCurve.from_spot_rates(l)
d = DateCurve(c, origin=today.to_date())

#d.domain
t9y = (today + '9y').to_date()
Df(d)(t9y), d.df(t9y)

In [ ]:
df = Df(d)
# df.curve.year_fraction(today + timedelta(days=365*3))
# df.curve.df(today)
df(today + timedelta(days=365*10))

In [ ]:
#d = DateCurve(c, origin=0)
d.curve[d.year_fraction(today + '1y')]

In [ ]:
curve=constant([0.0, 0.2518822724161533, 0.5037645448323066, 0.75564681724846, 0.999315537303217, 1.2511978097193703, 1.5030800821355237, 1.754962354551677, 1.998631074606434, 2.2505133470225873, 2.5023956194387407, 2.754277891854894, 2.9979466119096507, 3.249828884325804, 3.5017111567419574, 3.753593429158111, 4.0, 4.251882272416153, 4.503764544832307, 4.75564681724846, 4.999315537303217, 5.25119780971937, 5.503080082135524, 5.754962354551677, 5.9986310746064335, 6.250513347022587, 6.50239561943874, 6.754277891854894, 6.997946611909651, 7.2498288843258045, 7.501711156741957, 7.753593429158111, 8.0, 8.251882272416154, 8.503764544832306, 8.75564681724846, 8.999315537303216, 9.25119780971937, 9.503080082135524, 98.99520876112251], [0.02, 0.018, 0.016, 0.015, 0.017, 0.019, 0.02, 0.018, 0.016, 0.015, 0.017, 0.019, 0.02, 0.018, 0.016, 0.015, 0.017, 0.019, 0.02, 0.018, 0.016, 0.015, 0.017, 0.019, 0.02, 0.018, 0.016, 0.015, 0.017, 0.019, 0.02, 0.018, 0.016, 0.015, 0.017, 0.019, 0.02, 0.018, 0.016, 0.015])

dcf_curve = dcf.ShortRateCurve(curve.x_list, curve.y_list, forward_tenor=0.25, origin=0)
yc_curve = ShortRate(curve, cash_frequency=None, frequency=None)
dcf_curve._TIME_SHIFT = yc_curve.eps = eps = 1 /365

In [ ]:
yc_curve.zero(0.0027397260273972603)

In [ ]:
a = dcf_curve.get_zero_rate(0.0027397260273972603)
b = yc_curve.zero(0.0027397260273972603)
a, b

In [ ]:
a = dcf_curve.get_short_rate(0.0027397260273972603)
b = yc_curve.curve(0.0027397260273972603)
a, b

In [ ]:

self, start, stop = dcf_curve, 0.0, 0.0027378507871321013
print(f"{start=} {stop=}")
ir = self._get_compounding_rate(start, stop)
t = self.day_count(start, stop)
df = self._get_compounding_factor(start, stop)
print(f"{ir=} {t=} {df=}")
print(f"{self._get_compounding_rate(0, stop)=} {self._get_compounding_rate(0, start)=}")
print(f"{self._get_compounding_factor(0, stop)=} {self._get_compounding_factor(0, start)=}")

ir = yc_curve.zero(start, stop)
t = stop - start
df = yc_curve.df(start, stop)
print(f"{ir=} {t=} {df=}")

print(f"{yc_curve.curve(stop)=} {yc_curve.curve(start)=}")
print(f"{yc_curve.df(stop)=} {yc_curve.df(start)=}")

dcf_curve.get_short_rate(0.0027378507871321013), yc_curve.short(0.0027378507871321013)


In [ ]:
d = dcf.ZeroRateCurve(l.x_list, l.y_list, forward_tenor=0.25)
start, stop = x, y
d.get_discount_factor(x, y), d.get_cash_rate(x, y), d._get_compounding_factor(
    start, stop)

In [ ]:
domain = [1, 2, 3, 5, 7, 10, 15, 20, 30, 50]
l = linear([1, 2, 3, 5, 7, 50], [0.01, 0.05, 0.02, 0.03])
print(repr(l))
str(l), repr(l), l(2.5), l.x_list

In [ ]:
# c = NelsonSiegelSvensson.download(-1)
c = NelsonSiegelSvensson(beta0=1.9137, beta1=1.97448, beta2=-28.16468,
                         beta3=27.22348, tau1=4.6538, tau2=5.29223)
c

In [ ]:
NelsonSiegelSvenssonShortRate(**c.params)

In [ ]:
a = AssetPriceYield(.03, spot=40)
a, a.price_yield(1, 2), a.price(10)

In [ ]:
z = ZeroRate(l)
df = Df(z)
z.par(5, 10)

In [ ]:
c = Const(0.01)
z = ZeroRate(c, cash_frequency=2)
df = Df(z)

In [ ]:
T = 3
dates = list(x / z.cash_frequency for x in range(T * z.cash_frequency)) + [T]
sum(df(x) / z.cash_frequency for x in dates[1:])

In [ ]:
swap_annuity(z, dates)

In [ ]:
swap_annuity(z, T, cash_frequency=z.cash_frequency)

In [ ]:
annuity = Annuity(z)
annuity, annuity(T)

In [ ]:
swap_par_rate(z, T, cash_frequency=z.cash_frequency)

In [ ]:
par = Par(z)
par, par(T), par(T, 15), z.par(T), z.par(T, 15), z.par

In [ ]:
Cash(z), Cash(z)(T), z.cash(T), z.cash

In [ ]:
Df(z).cash(5), z.cash(5)

In [ ]:
#_ = plot(lin(0.25, 30, 0.25), z, Par(z), Cash(z), Cash(z, cash_frequency=1),legend=True)
_ = plot(lin(0.25, 25, 0.25), Cash(z), Cash(z, cash_frequency=1), Par(z),
         Par(z, cash_frequency=1), legend=True)

In [ ]:
zcurve = ZeroRate(0.01)
df = Df(zcurve)
zcurve, df, df(5, 10), zcurve.df(5, 10), df.zero(5, 10), zcurve(5, 10)

In [ ]:
zcurve = ZeroRate(0.01)
fwd = CashRate(ZeroRate(0.01), frequency=4)
swp = Par(zcurve, forward_curve=fwd)
swp

In [ ]:
df = Df(zcurve)
zcurve.cash(10), df.cash(10), Cash(df)(10)

In [ ]:
z = ZeroRate(c, frequency=1)

In [ ]:
c = NelsonSiegelSvensson.download(-1)
setattr(c, 'repr_repr', r'$i(t)$')
d = NelsonSiegelSvensson.download(-1)
d.short_rate = True
setattr(d, 'repr_repr', r'$r(t)$')

In [ ]:
ascii_plot(lin(0, 30, 0.025), c, d)

In [ ]:
_ = plot(lin(0, 30, 0.025), c, d, legend=True)